In [1]:
#  Environment and Ollama setup
# This cell loads environment variables, sets up the Ollama model and API URL, and prints them for verification.
import os
import requests
from dotenv import load_dotenv, find_dotenv

# Load environment variables (optional)
_ = load_dotenv(find_dotenv())

# Set Ollama model and API URL
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "gemma3:1b")
OLLAMA_API_URL = os.getenv("OLLAMA_API_URL", "http://localhost:11434")

def get_completion_from_messages(messages, model=OLLAMA_MODEL, temperature=0):
    """
    Generates responses from Ollama LLM using a list of messages (for chat-style prompts)
    """
    # Concatenate all messages into a single prompt string
    prompt = "\n".join([msg["content"] for msg in messages])
    url = f"{OLLAMA_API_URL}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "stream": False,
    }
    response = requests.post(url, json=payload)
    if response.status_code != 200:
        raise Exception(f"Ollama API error : {response.status_code} {response.text}")
    data = response.json()
    return data['response']

# Example usage
messages = [
    {"role": "user", "content": "Hello world"}
]

response = get_completion_from_messages(messages, model="gemma3:1b")
print(response)

Hello there! How can I help you today? 😊



In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [4]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11752\144666821.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=4) #increased number of results


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        self.model = model
        self.tools = {t.name: t for t in tools}
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_ai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_ai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
# Define a wrapper or use your function directly
def ollama_chat(messages, model="gemma3:1b", temperature=0):
    return get_completion_from_messages(messages, model=model, temperature=temperature)

In [9]:
messages = [{"role": "user", "content": "prompt"}]


In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ollama_chat(messages)  #reduce inference cost
abot = Agent(ollama_chat, [tool], system=prompt)

In [11]:
class OllamaModelWrapper:
    def __init__(self, model_name="gemma3:1b", temperature=0):
        self.model_name = model_name
        self.temperature = temperature

    def invoke(self, messages):
        # messages should be a list of LangChain message objects or dicts
        # Convert LangChain messages to dicts if needed
        prompt_msgs = []
        for msg in messages:
            if hasattr(msg, "content"):
                prompt_msgs.append({"role": "user", "content": msg.content})
            elif isinstance(msg, dict):
                prompt_msgs.append(msg)
        return get_completion_from_messages(prompt_msgs, model=self.model_name, temperature=self.temperature)

In [12]:
ollama_model = OllamaModelWrapper(model_name="gemma3:1b")
abot = Agent(ollama_model, [tool], system=prompt)


In [13]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

AttributeError: 'str' object has no attribute 'tool_calls'